In [3]:
!pip install openai
!pip install "openai[embedding_utils]"
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install langchain
!pip install matplotlib


[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
  Using cached matplotlib-3.7.1-cp39-cp39-macosx_11_0_arm64.whl (7.3 MB)
  Using cached fonttools-4.39.3-py3-none-any.whl (1.0 MB)
  Using cached kiwisolver-1.4.4-cp39-cp39-macosx_11_0_arm64.whl (63 kB)
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
  Using cached contourpy-1.0.7-cp39-cp39-macosx_11_0_arm64.whl (229 kB)
  Using cached cycler

In [4]:
import pandas as pd
import openai
import numpy as np
from langchain import OpenAI, LLMChain
from langchain.prompts import Prompt

from openai.embeddings_utils import cosine_similarity

ModuleNotFoundError: No module named 'plotly'

In [ ]:
%env OPENAI_API_KEY api_key

In [ ]:
# openai_api_key = ""

embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [ ]:
df = pd.read_csv("./docs/50_reviews_with_embeddings.csv")
df["embedding"] = df.embedding.apply(eval).apply(np.array)

In [ ]:
def get_embedding(text, model):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input=[text], model=model)["data"][0]["embedding"]

def search(source_df, query_str, depth):
    query_embedding = get_embedding(query_str, model=embedding_model)
    source_df["similarities"] = df.embedding.apply(lambda x: cosine_similarity(x, query_embedding))
    res = source_df.sort_values("similarities", ascending=False).head(depth)
    # print(res)
    return res

In [ ]:
with open("./training/master.txt", "rb") as master_template:
    prompt_template = master_template.read()

prompt = Prompt(template=prompt_template, input_variables=["history", "context", "question"])
llm_chain = LLMChain(prompt=prompt, llm=OpenAI(temperature=0.15, openai_api_key=openai_api_key))

In [ ]:
# TODO: implement according to semantic search recommendations https://github.com/openai/openai-cookbook/blob/main/examples/Semantic_text_search_using_embeddings.ipynb
def search_contexts_by_text_prompt(text_prompt):
    res = search(df, query_str=text_prompt, depth=10)
    combined_values_list = list(res.combined.values)
    prompt_contexts = []
    for idx, value in enumerate(combined_values_list):
        prompt_contexts.append(f'Context {idx}:\n{value}')
    return prompt_contexts

# print("###".join(combined_values_list))

history = []

# TODO: align with the official example https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb
def predict_answer(question_str):
    contexts = search_contexts_by_text_prompt(question_str)
    answer = llm_chain.predict(
        question=question_str,
        context="\n\n".join(contexts),
        # history=history
        history=[]
    )
    history.append(f'Human: {question_str}')
    history.append(f'Bot: {answer}')
    return answer

In [ ]:
question = "Whi are you?"
answer = predict_answer(question)
answer

In [ ]:
question = "What is Nectresse Sweetener?"
answer = predict_answer(question)
answer

In [ ]:
question = "What do you think about a hot cider?"
answer = predict_answer(question)
answer

In [ ]:
question = "Does hot cider taste good?"
answer = predict_answer(question)
answer